In [1]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
def corr_(df, n = 0.85, x = 1):
    '''
    
    Input - df: dataframe
            n: lower limit of correlation
            x: upper limit of correlation
            
    Output - dataframe with correlated pairs and correlation values
    
    '''
    df_cc=df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    df_cc['pairs'] = list(zip(df_cc.level_0, df_cc.level_1))
    df_cc.set_index(['pairs'], inplace = True)
    df_cc.drop(columns=['level_1', 'level_0'], inplace = True)
    df_cc.columns = ['cc']
    df_cc.drop_duplicates(inplace=True)
    
    return df_cc[(df_cc['cc'] > n) & (df_cc['cc'] < x)]

def find_outlier(df, feature):
    q1 = df[feature].quantile(0.25)
    q3 = df[feature].quantile(0.75)
    iqr = q3 - q1
    lower = q1-(iqr*1.5)
    upper = q3+(iqr*1.5)
    return iqr, lower, upper

def convert_game_clock(row):
    raw_game_clock = row['clock']
    quarter = int(row['quarter'])
    game_clock = raw_game_clock.split(':')
    minutes = game_clock[0]
    seconds = game_clock[1]
    seconds_left_in_quarter = int(seconds) + (int(minutes) * 60)
    
    if quarter == 1 or quarter == 3:
        return seconds_left_in_quarter + 900
    elif quarter == 2 or quarter == 4:
        return seconds_left_in_quarter
    
    
def make_table(data, groups, agg=['sum', 'count']):

    data = data.groupby(groups).agg(agg)['complete']
    data['pct'] = data['sum']/data['count']
    return data

# SportRadar

In [3]:
# establish connection to sportradar api
# need to enter private api_key
import http.client

api_key = api_key
conn = http.client.HTTPSConnection("api.sportradar.us")
conn.request("GET", f"/nfl/official/trial/v6/en/games/0e00303b-ee60-4cf4-ad68-48efbe53901d/pbp.json?api_key={api_key}")

res = conn.getresponse()
data = res.read()
json_object = json.loads(data)


In [4]:
# unpacking json object data 
df = pd.json_normalize(json_object, record_path=['periods'], 
                       meta=[['summary', 'season', 'year']], errors='ignore')
season = df['summary.season.year'][0]

In [5]:
# unpacking plays by quarter and combining all drives
q1 = pd.json_normalize(df['pbp'][0])
q2 = pd.json_normalize(df['pbp'][1])
q3 = pd.json_normalize(df['pbp'][2])
q4 = pd.json_normalize(df['pbp'][3])
quarters = [q1, q2, q3, q4]

drives = pd.concat(quarters, ignore_index=True)
drives = drives.loc[drives['type'] == 'drive']

In [7]:
# filtering pass plays from all plays
plays = []
for i, row in drives.iterrows():
    play = row['events']
    for n in play:
        plays.append(n)

# some pass plays missing pass_route data
passing_areas = ['Flat', 'Underneath Screen', 'WR Screen',
                 'Comeback', 'Curl', 'Cross', 'Go', 'Slant', 'Out']
plays = pd.DataFrame(plays)

# include pass_plays that include pass_route
pass_plays = plays.loc[(plays['play_type'] == 'pass') & (plays['pass_route'].isin(passing_areas))]
pass_plays.reset_index(inplace=True, drop=True)

### Sanity Check

In [8]:
# investigating pass plays that did not include pass_route data
penalties = plays.loc[(plays['play_type'] == 'pass') & ~(plays['pass_route'].isin(passing_areas))]

In [9]:
# offensive pass interference, noplay
penalties.loc[73]['details']

[{'category': 'pass_completion',
  'description': 'J.Garoppolo pass deep right complete to KC 21.',
  'sequence': 1,
  'direction': 'deep right',
  'yards': 42,
  'start_location': {'alias': 'SF', 'yardline': 45},
  'end_location': {'alias': 'KC', 'yardline': 21},
  'players': [{'id': '42de9d1d-0352-460b-9172-9452414fd7fd',
    'name': 'Jimmy Garoppolo',
    'jersey': '10',
    'position': 'QB',
    'sr_id': 'sr:player:828285',
    'role': 'pass'}]},
 {'category': 'pass_reception',
  'description': 'Catch made by G.Kittle at KC 21. Gain of 42 yards.',
  'sequence': 2,
  'yards': 42,
  'result': 'tackled',
  'start_location': {'alias': 'KC', 'yardline': 21},
  'end_location': {'alias': 'KC', 'yardline': 13},
  'players': [{'id': '2ada91b0-036e-454f-83c3-6d939ff584a9',
    'name': 'George Kittle',
    'jersey': '85',
    'position': 'TE',
    'sr_id': 'sr:player:1130029',
    'role': 'catch'}]},
 {'category': 'tackle',
  'description': 'Tackled by D.Sorensen at KC 13.',
  'sequence': 3,


In [10]:
# rollout at end of half, not counted towards stats
penalties.loc[171] 

type                                                                play
id                                  88eceb25-606e-4572-9229-d8404190f3af
sequence                                                          4091.0
reference                                                           4091
clock                                                              00:05
home_points                                                         31.0
away_points                                                         20.0
play_type                                                           pass
play_clock                                                          23.0
wall_clock                                     2020-02-03T03:10:28+00:00
description            (:05) (Shotgun) 15-P.Mahomes pass incomplete d...
alt_description        (:05) (Shotgun) P.Mahomes pass incomplete deep...
fake_punt                                                          False
fake_field_goal                                    

In [11]:
# getting stats from pass_plays
passing_stats = []
for i, item in pass_plays['statistics'].iteritems():
    stats = pd.Series(item[0])
    passing_stats.append(stats)
    
# getting additional details from pass_plays    
passing_details = []
for i, item in pass_plays['details'].iteritems():
    details = pd.Series(item[0])
    passing_details.append(details)
    

df = pd.DataFrame(passing_stats)
dfi = pd.DataFrame(passing_details)

df = pd.concat([df,dfi], axis=1)

# additional data in pass_plays
additional_columns = ['hash_mark', 'play_action', 'run_pass_option', 'blitz',
                      'qb_at_snap', 'players_rushed', 'pocket_location', 'pass_route']

df = df.merge(pass_plays[additional_columns], left_index=True, right_index=True)
df['season'] = season

In [13]:
# adding name/team for each qb
names = []
teams = []
for i, row in df.iterrows():

    name = row['player']['name']
    names.append(name)
    team = row['team']['alias']
    teams.append(team)
df['name'] = names
df['team'] = teams

## Cleaning Data Columns

In [14]:
df.shape

(70, 37)

In [15]:
# redundant data
dropping = ['player', 'description', 'players',
            'stat_type', 'attempt', 'category']
df.drop(columns=dropping, inplace=True)

In [17]:
# removing duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

In [18]:
# reordering columns
name_col = df.pop('name')
team_col = df.pop('team')
df.insert(0, 'name', name_col)
df.insert(1, 'team', team_col)

In [28]:
# export data
df.to_csv('./Data/first_game.csv')

---
## End of Pass Data from 1 Game
***

## Getting Game ID from Schedule

In [21]:
api_key = api_key
conn = http.client.HTTPSConnection("api.sportradar.us")

# 2020 schedule
conn.request("GET", f"/nfl/official/trial/v6/en/games/2020/REG/schedule.json?api_key={api_key}")
res = conn.getresponse()
data = res.read()



In [22]:
# convert schedule data to df
json_object = json.loads(data)
sch = pd.DataFrame(json_object)

{'id': '2eec467f-9ce1-484c-b58f-e11615bede68',
 'year': 2020,
 'type': 'REG',
 'name': 'REG',
 'weeks': [{'id': 'fd51f745-b7eb-4c01-8550-d9095aa5407d',
   'sequence': 1,
   'title': '1',
   'games': [{'id': '018556f9-1977-4a0b-8244-20cad15df9a4',
     'status': 'closed',
     'reference': '58178',
     'number': 11,
     'scheduled': '2020-09-13T20:05:00+00:00',
     'attendance': 0,
     'utc_offset': -5,
     'entry_mode': 'INGEST',
     'weather': 'Partly Cloudy Temp: 83 F, Humidity: 56%, Wind: NW 6 mph, gusts to 8 mph',
     'sr_id': 'sr:match:21980371',
     'venue': {'id': 'b87a1595-d3c8-48ea-8a53-0aab6378a64a',
      'name': 'Paul Brown Stadium',
      'city': 'Cincinnati',
      'state': 'OH',
      'country': 'USA',
      'zip': '45202',
      'address': 'One Paul Brown Stadium',
      'capacity': 65515,
      'surface': 'artificial',
      'roof_type': 'outdoor',
      'sr_id': 'sr:venue:8175',
      'location': {'lat': '39.095413', 'lng': '-84.516204'}},
     'home': {'id': 

In [24]:
# pull unique game_id from 2020 schedule
all_games = []
for i, item in sch['weeks'].iteritems():
    games = item['games']
    for n in range(len(games)):
        all_games.append(games[n])
    
all_games_info = pd.DataFrame(all_games)

all_games_info = all_games_info.loc[:, 'id']
all_games_id = list(all_games_info)

In [27]:
# add private api_key
# loop through all_games_id to get data from each game in 2020 season
api_key = api_key
conn = http.client.HTTPSConnection("api.sportradar.us")
game_data = []
for n, game in enumerate(all_games_id):
    if n % 10 == 0:
        print(f'Completed {n} games')
        
    conn.request("GET", f"/nfl/official/trial/v6/en/games/{game}/pbp.json?api_key={api_key}")
    res = conn.getresponse()
    
    data = res.read()
    game_data.append(data)


Completed 0 games


KeyboardInterrupt: 

In [ ]:
# byte data shorter than 10k is not full game
# either postponed or no game data
list_data = [d for d in game_data if len(d) > 10000] 

In [ ]:
def get_plays(data):
    '''
    Input - data: json api data from sportradar
    
    Output - cleaned df of qb stats from game
    '''
    
    obj = json.loads(data)
    df = pd.json_normalize(obj, record_path=['periods'],
                           meta=[['summary', 'season', 'year']], errors='ignore')
    season = df['summary.season.year'][0]

    q1 = pd.json_normalize(df['pbp'][0])
    q1['quarter'] = 1
    q2 = pd.json_normalize(df['pbp'][1])
    q2['quarter'] = 2
    q3 = pd.json_normalize(df['pbp'][2])
    q3['quarter'] = 3
    q4 = pd.json_normalize(df['pbp'][3])
    q4['quarter'] = 4
    quarters = [q1, q2, q3, q4]

    drives = pd.concat(quarters, ignore_index=True)
    drives = drives.loc[drives['type'] == 'drive']

    plays = []
    quarts = {1:[], 2:[], 3:[], 4:[]}

    for i, row in drives.iterrows():
        drive_id = row['events'][0]['id']
        quarter = row['quarter']
        quarts[quarter].append(drive_id)
        play = row['events']
        for n in play:
            plays.append(n)

    passing_areas = ['Flat', 'Underneath Screen', 'WR Screen',
                     'Comeback', 'Curl', 'Cross', 'Go', 'Slant', 'Out']
    non_play_event = ['period_end', 'timeout', 'game_over']

    plays = pd.DataFrame(plays)

    for k,v in quarts.items():
        for i, row in plays.iterrows():
            if row['id'] in v:
                plays.loc[i, 'quarter'] = k
    plays['quarter'].fillna(method='ffill', inplace=True)

    pass_plays = plays.loc[(plays['play_type'] == 'pass') & 
                           (plays['pass_route'].isin(passing_areas)) &
                           ~(plays['event_type'].isin(non_play_event))]
    pass_plays.reset_index(inplace=True, drop=True)

    passing_stats = []
    for i, item in pass_plays['statistics'].iteritems():
        stats = pd.Series(item[0])
        passing_stats.append(stats)


    passing_details = []
    for i, item in pass_plays['details'].iteritems():
        details = pd.Series(item[0])
        passing_details.append(details)

    down_info = []
    for i, row in pass_plays.iterrows():
        c = row['clock']
        q = row['quarter']
        d = row['start_situation']['down']
        y = row['start_situation']['yfd']
        down_info.append((c,q,d,y))


    df = pd.DataFrame(passing_stats)
    dfi = pd.DataFrame(passing_details)
    dfii = pd.DataFrame(down_info, 
                        columns=['clock', 'quarter', 'down', 'yards_to_first'])

    df = pd.concat([df,dfi,dfii], axis=1)

    # additional data in pass_plays
    additional_columns = ['hash_mark', 'play_action', 'run_pass_option', 'blitz',
                          'qb_at_snap', 'players_rushed', 'pocket_location', 'pass_route']

    df = df.merge(pass_plays[additional_columns], left_index=True, right_index=True)
    df['season'] = season
#     df['quarter'] = df['quarter'].astype('int')
    df = df.loc[df['stat_type'] == 'pass']

    names = []
    teams = []
    for i, row in df.iterrows():

        name = row['player']['name']
        names.append(name)
        team = row['team']['alias']
        teams.append(team)
    df['name'] = names
    df['team'] = teams

    dropping = ['player', 'description', 'players',
                'stat_type', 'attempt', 'category'] 
    df.drop(columns=dropping, inplace=True)
    df = df.loc[:, ~df.columns.duplicated()]

    name_col = df.pop('name')
    team_col = df.pop('team')
    df.insert(0, 'name', name_col)
    df.insert(1, 'team', team_col)

    return df

In [ ]:
# list_data = games actually played, does not include postponed/cancelled games (covid-19)

all_dfs = []
for n, game in enumerate(list_data):
    if n % 20 == 0:
        print(f'Completed {n+1} games')
    p = get_plays(game)
    all_dfs.append(p)

In [ ]:
# creating df combining all 2020 game data
df = pd.concat(all_dfs, ignore_index=True)
df.shape

In [ ]:
# export df to csv
df.to_csv('./Data/all_dfs.csv')